In [9]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.svm import SVR
from mtr_time_series import MTRTimeSeries

In [10]:
# Read the data
data = pd.read_csv('../../Data/all_stocks_5yr.csv')
data.head(5)

,Date,Open,High,Low,Close,Volume,Name
0,2012-08-13,92.29,92.59,91.74,92.40,2075391.0,MMM
1,2012-08-14,92.36,92.50,92.01,92.30,1843476.0,MMM
2,2012-08-15,92.00,92.74,91.94,92.54,1983395.0,MMM
3,2012-08-16,92.75,93.87,92.21,93.74,3395145.0,MMM
4,2012-08-17,93.93,94.30,93.59,94.24,3069513.0,MMM


### All the actives

In [11]:
# All actives
' '.join(np.unique(data.loc[:, 'Name']).tolist())

'A AAL AAP AAPL ABBV ABC ABT ACN ADBE ADI ADM ADP ADS ADSK AEE AEP AES AET AFL AGN AIG AIV AIZ AJG AKAM ALB ALGN ALK ALL ALLE ALXN AMAT AMD AME AMG AMGN AMP AMZN ANDV ANSS ANTM AON AOS APA APC APD APH ARE ARNC ATVI AVB AVGO AVY AWK AXP AYI AZO BA BAC BAX BBT BBY BCR BDX BEN BF.B BHF BHGE BIIB BK BLK BLL BMY BRK.B BSX BWA BXP C CA CAG CAH CAT CB CBG CBOE CBS CCI CCL CELG CERN CF CFG CHD CHK CHRW CHTR CI CINF CL CLX CMA CMCSA CME CMG CMI CMS CNC CNP COF COG COH COL COO COP COST COTY CPB CRM CSCO CSRA CSX CTAS CTL CTSH CTXS CVS CVX CXO D DAL DD DE DFS DG DGX DHI DHR DIS DISCA DISCK DISH DLPH DLR DLTR DOV DOW DPS DRE DRI DTE DUK DVA DVN DXC EA EBAY ECL ED EFX EIX EL EMN EMR EOG EQIX EQR EQT ES ESRX ESS ETFC ETN ETR EVHC EW EXC EXPD EXPE EXR F FAST FB FBHS FCX FDX FE FFIV FIS FISV FITB FL FLIR FLR FLS FMC FOX FOXA FRT FTI FTV GD GE GGP GILD GIS GLW GM GOOG GOOGL GPC GPN GPS GRMN GS GT GWW HAL HAS HBAN HBI HCA HCN HCP HD HES HIG HLT HOG HOLX HON HP HPE HPQ HRB HRL HRS HSIC HST HSY HUM IBM IC

### Now I'm going to select a single active to apply the MTR strategy (horizon predicting)

In [12]:
select_actv = 'MMM'

s_data = data[data['Name'] == select_actv]

### Counting the amount of examples for the active

In [ ]:
len(s_data)

1258

### Setup time serie

In [ ]:
time_serie = s_data['Close'].values.tolist()
time_serie[1:5]

[92.3, 92.54, 93.74, 94.24]

### Create and setup MTR evaluator

In [ ]:
mtr_ts = MTRTimeSeries(
    time_serie=time_serie,
    window_size=20,
    horizon_size=5,
    init_buffer_size=200
)

### Set the regressor for the SST method

In [ ]:
mtr_ts.set_regressor(RandomForestRegressor, {'n_estimators': 10, 'n_jobs': 6})
# mtr_ts.set_regressor(Lasso, {'alpha': 0.1})
# mtr_ts.set_regressor(ElasticNet, {})
# mtr_ts.set_regressor(SVR, {'kernel': 'poly', 'degree': 4, 'gamma': 'auto'})

### Iterate over time serie and get the predictions

In [ ]:
predictions = np.array(mtr_ts.iterate_fit_predict())

In [ ]:
predictions[1:10]

array([111.187, 111.578, 110.886, 111.142, 111.264, 110.46 , 110.349,
       110.209, 109.463])

### Do the same thing but applying an exponential decay smoothing

In [ ]:
mtr_ts_decay = MTRTimeSeries(
    time_serie=time_serie,
    window_size=20,
    horizon_size=5,
    init_buffer_size=200,
    use_exp_smoothing=True,
    gamma=0.5
)
mtr_ts_decay.set_regressor(RandomForestRegressor, {'n_estimators': 10, 'n_jobs': 6})
# mtr_ts_decay.set_regressor(Lasso, {'alpha': 0.1})
# mtr_ts_decay.set_regressor(ElasticNet, {})
# mtr_ts_decay.set_regressor(SVR, {'kernel': 'poly', 'degree': 4, 'gamma': 'auto'})
predictions_decay = np.array(mtr_ts_decay.iterate_fit_predict())

In [ ]:
pred_start = mtr_ts.init_prediction_index()
observations = np.array(time_serie[pred_start:(pred_start + len(predictions))])
plt.figure(figsize=(10,5))
plt.plot(observations, label='Observed')
plt.plot(predictions, c='r', label='Predicted')
plt.plot(predictions_decay, c='g', label='Exp Smoothing + Prediction')
plt.legend(fontsize=8)
plt.show()

### Error functions

In [ ]:
def MSE(obs, pred):
    return np.sum((obs - pred) ** 2)/len(obs)

def RMSE(obs, pred):
    return np.sqrt(np.sum((obs - pred) ** 2)/len(obs))

def RRMSE(obs, pred):
    num = np.sum((obs - pred) ** 2) + 1e-6
    den = np.sum((obs - np.mean(obs)) ** 2) + 1e-6
    return np.sqrt(num/den)

### Observed errors

In [ ]:
print('MSE: {}'.format(MSE(observations, predictions)))
print('RMSE: {}'.format(RMSE(observations, predictions)))
print('RRMSE: {}'.format(RRMSE(observations, predictions)))
print()
print('MSE-Decay: {}'.format(MSE(observations, predictions_decay)))
print('RMSE-Decay: {}'.format(RMSE(observations, predictions_decay)))
print('RRMSE-Decay: {}'.format(RRMSE(observations, predictions_decay)))